In [ ]:
###################
# set up the environment, nis, and image saving path
#####################
import os
from nis_util import *
from simple_detection import *

path_to_nis = 'C:\\Program Files\\NIS-Elements\\nis_ar.exe'
save_base_path = 'C:\\Users\\Nikon\\Documents\\David\\tmpOverview'

%matplotlib inline

In [ ]:
def do_scan(field, oc_overview, oc_detail, path_to_nis, save_base_path, prefix, suffix='.nd2', do_plot=True,
           z_range = 10, z_step=2, z_drive='Ti2 ZDrive'):
    # get field and directions 
    (left, right, top, bottom) = field
    direction = [1 if top<bottom else -1, 1 if left<right else -1]
    
    # do overview
    set_optical_configuration(path_to_nis, oc_overview)
    # get resolution
    (xres, yres, siz, mag) = get_resolution(path_to_nis)

    # get fov
    fov_x = xres * siz / mag
    fov_y = yres * siz / mag
    
    ov_path = os.path.join(save_base_path, prefix + '_overview' + suffix)    
    do_large_image_scan(path_to_nis, ov_path, left, right, top, bottom)
    
    print('finished overview, detecting wings...')
    
    # detect wings
    img = read_bf(ov_path)
    bboxes = detect_wings_simple(img, [top - fov_y * direction[0] / 2.0, left - fov_x * direction [1] / 2.0], [siz/mag,siz/mag], direction, plot=do_plot, layers=2)
    bboxes = [scale_bbox(bbox, expand_factor=.15) for bbox in bboxes]
    
    print('detected {} wings:'.format(len(bboxes)))
    idx = 0
    for bbox in bboxes:
        print('scanning wing {}: {}'.format(idx, bbox))
        
        (ymin, xmin, ymax, xmax) = bbox
        wing_path = os.path.join(save_base_path, prefix + '_wing' + str(idx) + suffix) 
        idx += 1
        
        set_optical_configuration(path_to_nis, oc_detail)
        pos = get_position(path_to_nis)
        
        fov = get_fov_from_res(get_resolution(path_to_nis))
        grid = gen_grid(fov, [xmin, ymin], [xmax, ymax], 0.15, True)
        
        nda = NDAcquisition(wing_path)
        nda.set_z(int(z_range/2), int(z_range/2), int(z_step), z_drive)
        nda.add_points(map(lambda x : (x[0], x[1], pos[2] - pos[3]), grid))
        nda.prepare(path_to_nis)
        nda.run(path_to_nis)

In [ ]:
slide_left = 'NG_Overview_024'
slide_mid = None #'NG_Overview_024'
slide_right = None#'NG_Overview_025'

do_scan_left  = slide_left != None
if do_scan_left:
    print('Scanning left scan.')
    do_scan((51462, 29950, -19500, 21000), 'DIA4x', 'DIA10x', path_to_nis, save_base_path, slide_left )
else:
    print('Skipping left slide.')

# mid slide
do_scan_mid  = slide_mid != None

if do_scan_mid:
    print('Scanning mid scan.')
    do_scan((11000, -10000, -19500, 21000), 'OC1', 'OC2', path_to_nis, save_base_path, slide_left )
else:
    print('Skipping middle slide.')

# right slide
do_scan_right  = slide_right != None
if do_scan_right:
    print('Scanning right scan.')
    do_scan((-28900, -50000, -19500, 21000), 'OC1', 'OC2', path_to_nis, save_base_path, slide_left )
else:
    print('Skipping right slide.')